<a href="https://colab.research.google.com/github/nisal2002/Activity-Club-Management-System/blob/master/inceptionV3_mango.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model

In [37]:
# Define paths
train_path = '/content/drive/MyDrive/Colab Notebooks/ripeness-mango/Training /Training 1'
test_path = '/content/drive/MyDrive/Colab Notebooks/ripeness-mango/Test'

In [38]:
# Image dimensions
img_size = (299, 299)

In [39]:
# Create an InceptionV3 base model
base_model = InceptionV3(input_shape=(299, 299, 3), include_top=False, weights='imagenet')

In [40]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [41]:
# Add custom layers for classification
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(base_model.input, x)

In [42]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
# Load and prepare training data
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=img_size,
                                                    batch_size=32,
                                                    class_mode='binary')

# Load and prepare testing data
test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=img_size,
                                                  batch_size=32,
                                                  class_mode='binary')

Found 0 images belonging to 0 classes.
Found 124 images belonging to 4 classes.


In [45]:
# # Print filenames and class indices for training data
# for filename, class_index in zip(train_generator.filenames, train_generator.classes):
#     print(f"Training - Filename: {filename}, Class Index: {class_index}")

# # Print filenames and class indices for testing data
# for filename, class_index in zip(test_generator.filenames, test_generator.classes):
#     print(f"Testing - Filename: {filename}, Class Index: {class_index}")

In [46]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

ValueError: ignored

In [ ]:
# Evaluate the model on the test set
accuracy = model.evaluate(test_generator)[1]
print(f"Test Accuracy: {accuracy}")